# 1. Parameters

In [ ]:
cases_dir = 'cases/unset'
metadata_file = 'input/metadata-subsample-pangolin.tsv'
build_tree = False

In [ ]:
from pathlib import Path
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

case_name = str(cases_dir_path.name)
index_path = cases_dir_path / 'index'

output_api_path = cases_dir_path / 'query-api.tsv'
output_cli_path = cases_dir_path / 'query-cli.tsv'

# 2. Benchmark command-line

In [ ]:
import pandas as pd
import genomics_data_index.api as gdi

def benchmark_cli_index(name: str, index_path: Path, build_tree: bool) -> pd.DataFrame:
    benchmark_commands = {
        'query hasa':               f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G"',
        'query isa':                f'gdi --project-dir {index_path} --ncores 1 query "isa:Switzerland/100108/2020"',
        'query --summary':          f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G" --summary',
        'query --features-summary': f'gdi --project-dir {index_path} --ncores 1 query --features-summary mutations',
        'list samples':             f'gdi --project-dir {index_path} --ncores 1 list samples',
    }
    
    if build_tree:
        benchmark_commands['query isin'] = f'gdi --project-dir {index_path} --ncores 1 query --reference-name NC_045512 "isin_5_substitutions:Switzerland/100108/2020"'
    
    db = gdi.GenomicsDataIndex.connect(index_path)
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    iterations = 10
    
    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_cli(name=name, kind_commands=benchmark_commands, number_samples=number_samples,
                                    number_features_no_unknown=number_features_no_unknown, number_features_all=number_features_all,
                                     iterations=iterations)

In [ ]:
cli_df = benchmark_cli_index(name=case_name, index_path=index_path, build_tree=build_tree)
cli_df.head(3)

In [ ]:
cli_df.to_csv(output_cli_path, sep='\t', index=False)

# 3. Test query API

## 3.1. Load (example) metadata

The simulated data is based off of real sample names and a real tree. So I can load up real metadata and attach it to a query (though the mutations and reference genome are all simulated).

In [ ]:
import pandas as pd

metadata_df = pd.read_csv(metadata_file, sep='\t')
metadata_df.head(2)

## 3.2. Define benchmark cases

In [ ]:
from typing import List
import genomics_data_index.api as gdi

def benchmark_api_index(name: str, index_path: Path, build_tree: bool) -> pd.DataFrame:
    db = gdi.GenomicsDataIndex.connect(index_path)
    q_no_join = db.samples_query(reference_name='NC_045512', universe='mutations')
    q_join = db.samples_query(reference_name='NC_045512', universe='mutations').join(metadata_df, sample_names_column='strain')

    q = q_join.hasa('hgvs_gn:NC_045512.2:S:p.D614G')
    r = q_join.hasa('hgvs_gn:NC_045512.2:N:p.R203K')
    
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    repeat = 10

    benchmark_cases = {
        'db.samples_query':      lambda: db.samples_query(reference_name='NC_045512', universe='mutations'),
        'q.join':                lambda: q_no_join.join(metadata_df, sample_names_column='strain'),
        'q.features_summary':    lambda: q_join.features_summary(),
        'q.features_comparison': lambda: q_join.features_comparison(sample_categories='lineage', categories_kind='dataframe', kind='mutations', unit='proportion'),
        'q.hasa':                lambda: q_join.hasa("hgvs_gn:NC_045512.2:N:p.R203K"),
        'q.isa':                 lambda: q_join.isa("Switzerland/100112/2020"),
        'q AND r':               lambda: q & r,
        'q.toframe':             lambda: q_join.toframe(),
        'q.summary':             lambda: q_join.summary(),
    }
    
    if build_tree:
        benchmark_cases['q.isin (distance)'] = lambda: q_join.isin("Switzerland/100108/2020", kind='distance', distance=5, units='substitutions')
        benchmark_cases['q.isin (mrca)'] = lambda: q_join.isin(["Switzerland/100108/2020", "FR993751"], kind='mrca')

    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_api(name=name, kind_functions=benchmark_cases,
                                       number_samples=number_samples,
                                       number_features_no_unknown=number_features_no_unknown,
                                       number_features_all=number_features_all,
                                       repeat=repeat)

## 3.3. Benchmark reads index

In [ ]:
api_df = benchmark_api_index(name=case_name, index_path=index_path, build_tree=build_tree)
api_df.head(5)

In [ ]:
api_df.to_csv(output_api_path, sep='\t', index=False)